In [1]:
import time
import numpy as np
import pandas as pd

from tqdm import tqdm


In [2]:
def read_data(route):
    data = pd.read_csv(route)
    data = data[data['ts'] != 'ts']
    data['ts'] = data['ts'].astype('int64')
    # data['time_index'] = ((data['ts'] - data['ts'][0]) / 300) * 5
    
    return data

In [3]:
# 두 데이터 프레임을 ts 기준으로 병합 (하루전가격 기준으로 병합)
기상실측데이터1 = read_data('../OIBC_2024_DATA/data/기상실측데이터_1.csv')
기상실측데이터2 = read_data('../OIBC_2024_DATA/data/기상실측데이터_2.csv')
기상실측데이터 = pd.concat([기상실측데이터1, 기상실측데이터2], ignore_index=True)

# location에 대한 one-hot encoding
# ['Ilgwa-ri' 'Geumak-ri' 'Yongsu-ri' 'Hacheon-ri'
# 'Cheonji-dong' 'Songdang-ri' 'Bonggae-dong' 'Gwangryeong-ri' 'Sangmo-ri']
locations = 기상실측데이터['location'].unique()
기상실측데이터 = pd.get_dummies(기상실측데이터, columns=['location'])
기상실측데이터 = 기상실측데이터.fillna(0)

기상실측데이터

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Cheonji-dong,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri
0,1709222220,6.9,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,False,True,False,False,False,False
1,1709222580,7.1,2.7,2.7,68.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,False,True,False,False,False,False
2,1709226120,6.3,0.9,0.9,64.0,-0.1,338.0,22.0,22.0,0.0,...,False,False,False,False,False,True,False,False,False,False
3,1709229420,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,...,False,False,False,False,False,True,False,False,False,False
4,1709229840,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98663,1729600283,21.96,0,0,0,0,270.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
98664,1729602067,19.96,0,0,0,0,300.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
98665,1729603843,19.96,0,0,0,0,290.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
98666,1729605674,20.29,0,0,0,0,299.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False


In [4]:
하루전가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_하루전가격.csv')
# 하루전가격 = 하루전가격.drop(0)  # 첫 데이터는 학습 데이터가 없어서 제거
하루전가격

,ts,하루전가격(원/kWh)
0,1709218800,107.39
1,1709222400,107.39
2,1709226000,95.30
3,1709229600,87.89
4,1709233200,0.00
...,...,...
5660,1729594800,156.68
5661,1729598400,152.54
5662,1729602000,106.92
5663,1729605600,150.97


In [5]:
data = pd.merge(기상실측데이터, 하루전가격, on="ts", how="outer")

# location에 따른 interpolation
for location in locations:
    rows = data[data[f'location_{location}'] == True]
    rows = rows.interpolate(method='linear')
    # print(type(rows), rows[0])
    data = pd.concat([data, rows], ignore_index=True)
    
# nan 가진 데이터 제거 
# data = data.dropna(axis=0)

# ts를 기준으로 sort
data = data.sort_values('ts')
data.reset_index(inplace=True)
data = data.drop(columns=['index'])

data

/var/folders/f1/1vljqzq56clcfl2wk62vtd840000gq/T/ipykernel_60097/791625081.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  rows = rows.interpolate(method='linear')
/var/folders/f1/1vljqzq56clcfl2wk62vtd840000gq/T/ipykernel_60097/791625081.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  rows = rows.interpolate(method='linear')
/var/folders/f1/1vljqzq56clcfl2wk62vtd840000gq/T/ipykernel_60097/791625081.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  rows = rows.interpolate(method='linear')
/var/folders/f1/1vljqzq56clcfl2wk62vtd840000gq/T/ipykernel_60097/791625081.py:6: FutureWarning: DataFrame.interpolate 

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh)
0,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.39
1,1709218850,3.23,0,0,0,0,343.0,0,0,0,...,False,False,False,False,False,False,False,False,False,NaN
2,1709218850,3.23,0,0,0,0,343.0,0,0,0,...,False,False,False,False,False,False,False,False,False,NaN
3,1709218851,7.18,0,0,0,0,341.0,0,0,0,...,False,False,False,False,False,True,False,False,False,NaN
4,1709218851,7.18,0,0,0,0,341.0,0,0,0,...,False,False,False,False,False,True,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202938,1729608960,20.0,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,False,NaN
202939,1729608960,20.0,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,False,NaN
202940,1729608960,18.8,14.9,14.9,100.0,18.8,293.0,69.2,69.2,0.0,...,False,False,False,False,False,False,False,False,True,128.88
202941,1729608960,18.8,14.9,14.9,100.0,18.8,293.0,69.2,69.2,0.0,...,False,False,False,False,False,False,False,False,True,NaN


In [6]:
from tqdm import tqdm


# 가격 interpolation
# interpolation_data = data[['ts', '하루전가격(원/kWh)']]
# interpolation_data.drop_duplicates(subset='ts', keep='first')
# interpolation_data.set_index('ts', inplace=True)
# interpolation_data['하루전가격(원/kWh)'] = interpolation_data['하루전가격(원/kWh)'].interpolate()
# interpolation_data.reset_index(inplace=True)

def get_interpolation_value(data, row_index, column_name):
    if pd.notna(data.iloc[row_index, :][column_name]):
        return data.iloc[row_index, :][column_name]
    # 앞쪽에서 가장 가까운 NaN이 아닌 값의 인덱스 찾기
    previous_non_nan_index = data[column_name][:row_index].last_valid_index()
    
    # 뒤쪽에서 가장 가까운 NaN이 아닌 값의 인덱스 찾기
    next_non_nan_index = data[column_name][row_index + 1:].first_valid_index()
    # print(row_index, previous_non_nan_index, next_non_nan_index)
    
    if previous_non_nan_index is not None and next_non_nan_index is not None:
    # print(previous_non_nan_index, next_non_nan_index)
        time1 = data.iloc[previous_non_nan_index, :]['ts']
        time2 = data.iloc[row_index, :]['ts']
        time3 = data.iloc[next_non_nan_index, :]['ts']
    
        column1 = data.iloc[previous_non_nan_index, :][column_name]
        column3 = data.iloc[next_non_nan_index, :][column_name]
        # print(time1, time2, time3, column1)
        # time.sleep(5)
        
        # print(d1, d2)
        # 1000 1100 1200 2000
        # 1    x    x    10
        if time3 > time1:
            diff = (time2 - time1) / (time3 - time1)
            return column1 + diff * (column3 - column1)
        else:
            return column1
    elif next_non_nan_index is not None:
        column3 = data.iloc[next_non_nan_index, :][column_name]
        return column3
    else:
        column1 = data.iloc[previous_non_nan_index, :][column_name]
        return column1
        
def interpolate_all(old_data, column_name):
    new_data = old_data.copy()
    new_data.reset_index(inplace=True)
    new_data.drop(columns=['index'], inplace=True)
    
    interpolation_values_price = []
    for index, row in new_data.iterrows():
        # if not isinstance(row['하루전가격(원/kWh)'], float):
    
        res = get_interpolation_value(new_data, index, column_name)
        # print('index: {}, {}, {}'.format(index, row['ts'], res))
        interpolation_values_price.append(res)
        
    interpolation_values_price = pd.Series(interpolation_values_price)
    new_data[column_name] = interpolation_values_price
    return new_data


interpolation_data = data.copy()

# 모든 컬럼에 대해 결측치 보간
# for column in tqdm(interpolation_data.columns, ncols=75):
#     if not 'location' in column and column != 'ts':
#         interpolation_data[column] = interpolation_data[column].astype('float64')
#         interpolation_data = interpolate_all(interpolation_data, column)
interpolation_data['하루전가격(원/kWh)'] = interpolation_data['하루전가격(원/kWh)'].astype('float64')
interpolation_data = interpolate_all(data, '하루전가격(원/kWh)')

# 첫 컬럼은 제거
interpolation_data = interpolation_data.drop(0)

# 마지막 값 제거
# interpolation_data.drop(interpolation_data.index[-1], inplace=True)
    
# (1709222454-1709222400)/(1709226000-1709222400)
# 54/3600 = 0.015
# ts는 여러 지역의 센서가 동시에 캡쳐할 수도 있기에 같은 ts가 여러 개 나올 수 있다.
interpolation_data

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh)
1,1709218850,3.23,0,0,0,0,343.0,0,0,0,...,False,False,False,False,False,False,False,False,False,107.39
2,1709218850,3.23,0,0,0,0,343.0,0,0,0,...,False,False,False,False,False,False,False,False,False,107.39
3,1709218851,7.18,0,0,0,0,341.0,0,0,0,...,False,False,False,False,False,True,False,False,False,107.39
4,1709218851,7.18,0,0,0,0,341.0,0,0,0,...,False,False,False,False,False,True,False,False,False,107.39
5,1709218856,8.03,0,0,0,0,352.0,0,0,0,...,True,False,False,False,False,False,False,False,False,107.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202938,1729608960,20.0,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,False,128.88
202939,1729608960,20.0,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,False,128.88
202940,1729608960,18.8,14.9,14.9,100.0,18.8,293.0,69.2,69.2,0.0,...,False,False,False,False,False,False,False,False,True,128.88
202941,1729608960,18.8,14.9,14.9,100.0,18.8,293.0,69.2,69.2,0.0,...,False,False,False,False,False,False,False,False,True,128.88


In [7]:
data.to_csv('train_dataset.csv', index=False)
interpolation_data.to_csv('train_dataset_inter.csv', index=False)

In [8]:
하루전가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_하루전가격.csv')
실시간가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_실시간가격.csv')

실시간가격 = 실시간가격.drop(columns=['실시간 임시 가격(원/kWh)'])
하루전_실시간 = pd.merge(하루전가격, 실시간가격, on='ts', how='inner')

하루전_실시간.to_csv('days.csv', index=False)